In [1]:
import os
import yaml
import pytest
import pandas as pd

from network_wrangler import ProjectCard

from update_registry import update_registry
from methods_io import read_project_cards
from methods_add_cards import add_cards_to_registry
from methods_add_cards import _make_available

## defaults

In [2]:
CARD_DIR = os.path.join(".", "reference_projects")
REGISTRY_FILE = "registry.csv"
CONFIG_FILE = "registry_config.yml"

In [3]:
input_reg_df = pd.read_csv(REGISTRY_FILE)
input_reg_df

,type,id,project_added
0,node,1001,Project A
1,node,1002,Project A
2,link,501,Project A
3,node,1003,Project B
4,node,1004,Project B
5,link,502,Project B


## test methods

In [4]:
card_file_list = read_project_cards(CARD_DIR)
card_file_list

[(<network_wrangler.projectcard.ProjectCard at 0x7ffbee323d10>,
  './reference_projects/project_B.yml'),
 (<network_wrangler.projectcard.ProjectCard at 0x7ffbee47d050>,
  './reference_projects/project_A.yml')]

In [5]:
update_registry(
        CONFIG_FILE, REGISTRY_FILE, "delete_me.csv", CARD_DIR, False,
    )

## test 01

In [14]:
input_file = "test_input_registry.csv"
output_file = "test_update_registry.csv"

data = []
input_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
input_df.to_csv(input_file, index=False)

update_registry(
        config_file="registry_config.yml",
        input_reg_file=input_file,
        output_reg_file=output_file,
        card_dir=os.path.join(".", "reference_projects"),
        write_card_updates=False,
)

data = [
        ["node", 1001, "Project B"],
        ["node", 1002, "Project B"],
        ["node", 1003, "Project A"],
        ["node", 1004, "Project A"],
        ["link", 501, "Project B"],
        ["link", 502, "Project A"],
]
    
target_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
target_df = target_df.sort_values(by=["type", "id"]).reset_index(drop=True)

outcome_df = pd.read_csv(output_file)
outcome_df = outcome_df.sort_values(by=["type", "id"]).reset_index(drop=True)

# os.remove(input_file)
# os.remove(output_file)

In [15]:
target_df

,type,id,project_added
0,link,501,Project B
1,link,502,Project A
2,node,1001,Project B
3,node,1002,Project B
4,node,1003,Project A
5,node,1004,Project A


In [16]:
outcome_df

,type,id,project_added
0,link,501,Project B
1,link,502,Project A
2,node,1001,Project B
3,node,1002,Project B
4,node,1003,Project A
5,node,1004,Project A


In [17]:
target_df.equals(outcome_df)

True

## test 02

In [18]:
output_file = "test_update_registry.csv"
temp_file = "temp_registry.csv"
data = [
        ["node", 1001, "Project Z"],
        ["node", 1002, "Project Z"],
        ["link", 501, "Project Z"],
    ]
temp_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
temp_df.to_csv(temp_file, index=False)

update_registry(
        config_file="registry_config.yml",
        input_reg_file=temp_file,
        output_reg_file=output_file,
        card_dir=os.path.join(".", "reference_projects"),
        write_card_updates=False,
    )

data = [
        ["node", 1001, "Project Z"],
        ["node", 1002, "Project Z"],
        ["link", 501, "Project Z"],
        ["node", 1003, "Project B"],
        ["node", 1004, "Project B"],
        ["node", 1005, "Project A"],
        ["node", 1006, "Project A"],
        ["link", 502, "Project B"],
        ["link", 503, "Project A"],
    ]
target_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
target_df = target_df.sort_values(by=["type", "id"]).reset_index(drop=True)

outcome_df = pd.read_csv(output_file)
outcome_df = outcome_df.sort_values(by=["type", "id"]).reset_index(drop=True)

In [19]:
target_df

,type,id,project_added
0,link,501,Project Z
1,link,502,Project B
2,link,503,Project A
3,node,1001,Project Z
4,node,1002,Project Z
5,node,1003,Project B
6,node,1004,Project B
7,node,1005,Project A
8,node,1006,Project A


In [20]:
outcome_df

,type,id,project_added
0,link,501,Project Z
1,link,502,Project B
2,link,503,Project A
3,node,1001,Project Z
4,node,1002,Project Z
5,node,1003,Project B
6,node,1004,Project B
7,node,1005,Project A
8,node,1006,Project A


In [ ]:
target_df.equals(outcome_df)

## test 03
make sure procedures are not recursive. if project name is in the repository, take no action.

In [ ]:
input_file = "test_input_registry.csv"
    output_file = "test_update_registry.csv"

    data = [
        ["node", 1001, "Project B"],
        ["node", 1002, "Project B"],
        ["node", 1003, "Project A"],
        ["node", 1004, "Project A"],
        ["link", 501, "Project B"],
        ["link", 502, "Project A"],
    ]
    input_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
    input_df.to_csv(input_file)

    update_registry(
        config_file="registry_config.yml",
        input_reg_file=input_file,
        output_reg_file=output_file,
        card_dir=os.path.join(".", "reference_projects"),
        write_card_updates=False,
    )

    data = [
        ["node", 1001, "Project B"],
        ["node", 1002, "Project B"],
        ["node", 1003, "Project A"],
        ["node", 1004, "Project A"],
        ["link", 501, "Project B"],
        ["link", 502, "Project A"],
    ]
    target_i_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
    target_i_df = target_i_df.sort_values(by=["type", "id"]).reset_index(drop=True)

    data = [
        ["node", 1001, "Project A"],
        ["node", 1002, "Project A"],
        ["node", 1003, "Project B"],
        ["node", 1004, "Project B"],
        ["link", 501, "Project A"],
        ["link", 502, "Project B"],
    ]
    target_ii_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
    target_ii_df = target_ii_df.sort_values(by=["type", "id"]).reset_index(drop=True)

    outcome_df = pd.read_csv(output_file)
    outcome_df = outcome_df.sort_values(by=["type", "id"]).reset_index(drop=True)

    os.remove(input_file)
    os.remove(output_file)

    assert (
        target_i_df.equals(outcome_df) is True
        or target_ii_df.equals(outcome_df) is True
    )